In [38]:
import requests
import pandas as pd
from io import BytesIO
import zipfile
import io
import os
import datetime
from pathlib import Path
import numpy as np
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime
import timeit


In [39]:
symbol = '"NIFTY"'
expiry = "'27-Aug-2020'"
engine = create_engine('sqlite:///C:\\Users\\Tathagat\\Desktop\\BhavCopy\\SqLite\\foo.db' , echo=False)
backPeriod = 5

In [40]:
query = "SELECT * from users WHERE SYMBOL=" + symbol + "and EXPIRY_DT=" + expiry + "and (INSTRUMENT='OPTIDX' or INSTRUMENT='OPTSTK') "
temp = engine.execute(query).fetchall()
temp = pd.DataFrame(temp)
columnNames = engine.execute(" PRAGMA table_info(users) ")
columnNames = pd.DataFrame(columnNames)
columnNames = columnNames[1]
temp = temp.rename(columns=columnNames)
temp = temp.drop(temp.columns[[0,-1]] , axis=1)


In [41]:
temp['TIMESTAMP'] = temp['TIMESTAMP'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))
temp['EXPIRY_DT'] = temp['EXPIRY_DT'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))
temp = temp.sort_values('TIMESTAMP')
z = {}

In [42]:
start = timeit.default_timer()

for i in temp.index :
    x = temp.loc[i]
    expiry = x['EXPIRY_DT']
    strike = x['STRIKE_PR']
    option = x['OPTION_TYP']
    key = (expiry,strike,option)
    if key in z.keys() :
        z[key].append(x)
    else :
        z[key] = []
        z[key].append(x)
        
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  5.042613899999992


In [43]:
finalResult = []
for key in z.keys() :
    x = z[key]
    x = pd.DataFrame(x)
    index = x.index
    total = 0
    flag=-1
    prevAvg = 0
    prevClose = 0
    x['prevAvg'] = 0
    x['prevClose'] = 0
    x['currentAvg'] = 0
    x['currentClose'] = x['CLOSE']
    for idx in range(len(index)) :
        if idx>=backPeriod :
            average = total/backPeriod
            if x.loc[index[idx] , 'CLOSE'] < average :
                flag = 0
            
            if flag==0 and x.loc[index[idx] , 'CLOSE'] > average :
                flag=1
                x.loc[index[idx] , 'prevAvg'] = prevAvg
                x.loc[index[idx] , 'currentAvg'] = average
                x.loc[index[idx] , 'prevClose'] = prevClose
                finalResult.append(x.loc[index[idx]])
            
            total = total + x.loc[index[idx] , 'CLOSE']
            total = total - x.loc[index[idx - backPeriod] , 'CLOSE']
            prevAvg = average
            prevClose = x.loc[index[idx] , 'CLOSE']
        else :
            total = total + x.loc[index[idx] , 'CLOSE']
        
    
    

In [44]:
finalResult = pd.DataFrame(finalResult)

In [45]:
finalResult

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,prevAvg,prevClose,currentAverage,currentClose
3400,OPTIDX,NIFTY,2020-08-27,8000.0,CE,2400.00,2411.00,2400.00,2406.00,2406.00,3,23.40,2250,-75,2020-07-01,2319.27,2239.95,2274.98,2406.00
5440,OPTIDX,NIFTY,2020-08-27,8000.0,CE,2782.40,2962.05,2782.40,2933.45,2933.45,16,130.49,2550,-225,2020-07-17,2703.97,2635.00,2685.68,2933.45
6730,OPTIDX,NIFTY,2020-08-27,8000.0,CE,3140.00,3300.60,3140.00,3293.00,3293.00,365,3070.92,63300,24075,2020-07-28,3109.19,3104.75,3131.14,3293.00
7890,OPTIDX,NIFTY,2020-08-27,8000.0,CE,3143.35,3146.95,3064.70,3103.75,3103.75,142,1182.96,232875,-3000,2020-08-05,3106.00,3095.85,3066.57,3103.75
9636,OPTIDX,NIFTY,2020-08-27,8000.0,CE,3310.70,3410.00,3290.00,3392.00,3392.00,209,1774.83,126450,-12300,2020-08-18,3290.84,3286.80,3290.23,3392.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219,OPTIDX,NIFTY,2020-08-27,12600.0,PE,1326.40,1326.40,1275.70,1275.70,1278.00,4,41.66,2850,-225,2020-08-20,1242.10,1147.00,1218.71,1275.70
9633,OPTIDX,NIFTY,2020-08-27,12700.0,PE,1495.65,1495.65,1449.50,1449.50,1439.15,2,21.25,10500,0,2020-08-17,1409.43,1372.50,1390.35,1449.50
10221,OPTIDX,NIFTY,2020-08-27,12700.0,PE,1359.90,1408.90,1350.35,1386.80,1377.70,5,52.81,10500,0,2020-08-20,1389.20,1338.30,1384.27,1386.80
9441,OPTIDX,NIFTY,2020-08-27,12800.0,PE,1447.10,1648.00,1415.85,1625.00,1625.00,15,160.78,8775,-300,2020-08-14,1525.55,1490.00,1499.15,1625.00
